In [1]:
%%capture
import torch
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

!pip install torch_geometric
from torch_geometric.data import download_url, extract_zip

Download Movielens Dataset Small.

In [2]:
url = 'https://files.grouplens.org/datasets/movielens/ml-latest-small.zip'
extract_zip(download_url(url,'.'),'.')

Extracting ./ml-latest-small.zip


In [3]:
# Importing the datasets into variables movies_df and rating_df
movies_df = pd.read_csv('./ml-latest-small/movies.csv')
rating_df = pd.read_csv('./ml-latest-small/ratings.csv')

In [4]:
movies_df.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy

In [5]:
rating_df.head()

userId  movieId  rating  timestamp
0       1        1     4.0  964982703
1       1        3     4.0  964981247
2       1        6     4.0  964982224
3       1       47     5.0  964983815
4       1       50     5.0  964982931

Now we see how many unique movies and Users are there in the database.

In [ ]:
USERS = len(rating_df['userId'].unique())
MOVIES = len(movies_df['movieId'].unique())
print("Number of unique Users : ", USERS)
print("Number of unique Movies : ", MOVIES)

Number of unique Users :  610
Number of unique Movies :  9742


In [7]:
genre_entries = list(movies_df['genres'].unique())

print("Number of unique genre_movie_entries: ", len(genre_entries), genre_entries)

Number of unique genre_movie_entries:  951 ['Adventure|Animation|Children|Comedy|Fantasy', 'Adventure|Children|Fantasy', 'Comedy|Romance', 'Comedy|Drama|Romance', 'Comedy', 'Action|Crime|Thriller', 'Adventure|Children', 'Action', 'Action|Adventure|Thriller', 'Comedy|Horror', 'Adventure|Animation|Children', 'Drama', 'Action|Adventure|Romance', 'Crime|Drama', 'Drama|Romance', 'Action|Comedy|Crime|Drama|Thriller', 'Comedy|Crime|Thriller', 'Crime|Drama|Horror|Mystery|Thriller', 'Drama|Sci-Fi', 'Children|Drama', 'Adventure|Drama|Fantasy|Mystery|Sci-Fi', 'Mystery|Sci-Fi|Thriller', 'Children|Comedy', 'Drama|War', 'Action|Crime|Drama', 'Action|Adventure|Fantasy', 'Comedy|Drama|Thriller', 'Mystery|Thriller', 'Animation|Children|Drama|Musical|Romance', 'Crime|Mystery|Thriller', 'Adventure|Drama', 'Drama|Thriller', 'Comedy|Crime', 'Action|Sci-Fi|Thriller', 'Action|Comedy|Horror|Thriller', 'Comedy|Drama', 'Documentary', 'Action|Crime|Drama|Thriller', 'Crime|Drama|Romance', 'Action|Adventure|Drama'

Splitting up the genre_entries into the specific genre options

In [8]:
genres_list = [sublist.split('|') for sublist in genre_entries]

print("Genres List: ", genres_list)

# convert to lowecase so there is feature parody
genres_list = [genre.lower() for sublist in genres_list for genre in sublist]

unique_genres = list(set(genres_list))

unique_genres.sort()

print(unique_genres)

Genres List:  [['Adventure', 'Animation', 'Children', 'Comedy', 'Fantasy'], ['Adventure', 'Children', 'Fantasy'], ['Comedy', 'Romance'], ['Comedy', 'Drama', 'Romance'], ['Comedy'], ['Action', 'Crime', 'Thriller'], ['Adventure', 'Children'], ['Action'], ['Action', 'Adventure', 'Thriller'], ['Comedy', 'Horror'], ['Adventure', 'Animation', 'Children'], ['Drama'], ['Action', 'Adventure', 'Romance'], ['Crime', 'Drama'], ['Drama', 'Romance'], ['Action', 'Comedy', 'Crime', 'Drama', 'Thriller'], ['Comedy', 'Crime', 'Thriller'], ['Crime', 'Drama', 'Horror', 'Mystery', 'Thriller'], ['Drama', 'Sci-Fi'], ['Children', 'Drama'], ['Adventure', 'Drama', 'Fantasy', 'Mystery', 'Sci-Fi'], ['Mystery', 'Sci-Fi', 'Thriller'], ['Children', 'Comedy'], ['Drama', 'War'], ['Action', 'Crime', 'Drama'], ['Action', 'Adventure', 'Fantasy'], ['Comedy', 'Drama', 'Thriller'], ['Mystery', 'Thriller'], ['Animation', 'Children', 'Drama', 'Musical', 'Romance'], ['Crime', 'Mystery', 'Thriller'], ['Adventure', 'Drama'], ['Dr

In [9]:
print("Number of unique genres: ", len(unique_genres))

# Adding the unique genres as columns to the movies dataframe
for genre in unique_genres:
    movies_df[genre] = movies_df['genres'].transform(lambda x: int(genre in x))

movies_df.head()

Number of unique genres:  20


movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  (no genres listed)  action  \
0  Adventure|Animation|Children|Comedy|Fantasy                   0       0   
1                   Adventure|Children|Fantasy                   0       0   
2                               Comedy|Romance                   0       0   
3                         Comedy|Drama|Romance                   0       0   
4                                       Comedy                   0       0   

   adventure  animation  children  comedy  crime  ...  film-noir  horror  \
0          0          0         0       0      0  ...          0       0   
1          0          0         0       0      0  ...          0       0   
2          0          0         0       0      0  ...          0       0   
3          0          0         0       0      0  ...          0       0   
4          0          0         0       0      0  ...          0       0   

   imax  musical  mystery  romance  sci-fi  thriller  war  western  
0     0        0        0        0       0         0    0        0  
1     0        0        0        0       0         0    0        0  
2     0        0        0        0       0         0    0        0  
3     0        0        0        0       0         0    0        0  
4     0        0        0        0       0         0    0        0  

[5 rows x 23 columns]

In [10]:
# Adding a 1 if the "genres" column contains the genre, 0 otherwise
for index, genre_list in enumerate(movies_df['genres']):

    genre_list_cleaned = genre_list.split('|')
    genre_list_cleaned = [genre.lower() for genre in genre_list_cleaned]
    # print("Index: ", index, "Genre List: ", genre_list, "Genre List Cleaned: ", genre_list_cleaned)

    for genre in unique_genres:
        if genre in genre_list_cleaned:
            movies_df.at[index, genre] = 1
        else:
            movies_df.at[index, genre] = 0


movies_df.head()

movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  (no genres listed)  action  \
0  Adventure|Animation|Children|Comedy|Fantasy                   0       0   
1                   Adventure|Children|Fantasy                   0       0   
2                               Comedy|Romance                   0       0   
3                         Comedy|Drama|Romance                   0       0   
4                                       Comedy                   0       0   

   adventure  animation  children  comedy  crime  ...  film-noir  horror  \
0          1          1         1       1      0  ...          0       0   
1          1          0         1       0      0  ...          0       0   
2          0          0         0       1      0  ...          0       0   
3          0          0         0       1      0  ...          0       0   
4          0          0         0       1      0  ...          0       0   

   imax  musical  mystery  romance  sci-fi  thriller  war  western  
0     0        0        0        0       0         0    0        0  
1     0        0        0        0       0         0    0        0  
2     0        0        0        1       0         0    0        0  
3     0        0        0        1       0         0    0        0  
4     0        0        0        0       0         0    0        0  

[5 rows x 23 columns]

In [11]:
print(list(movies_df.columns))

['movieId', 'title', 'genres', '(no genres listed)', 'action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary', 'drama', 'fantasy', 'film-noir', 'horror', 'imax', 'musical', 'mystery', 'romance', 'sci-fi', 'thriller', 'war', 'western']


In [13]:
# Merging the two dataframes on the movieId column
final_df = rating_df.merge(movies_df, on='movieId')

# Need to scale all the ratings to be between 0 and 1
final_df['rating'] = final_df['rating'] / 5.0

# Splitting the final_df into training and testing data
# Cannot use train_test_split because it will split the data randomly and we need to only split along movieID
# Otherwise, the same movie will already be in the model when we are trying to recommend other movies
number_of_movies = len(final_df['movieId'].unique())
print("Number of unique movies: ", number_of_movies)

final_df.head()

Number of unique movies:  9724


userId  movieId  rating   timestamp             title  \
0       1        1     0.8   964982703  Toy Story (1995)   
1       5        1     0.8   847434962  Toy Story (1995)   
2       7        1     0.9  1106635946  Toy Story (1995)   
3      15        1     0.5  1510577970  Toy Story (1995)   
4      17        1     0.9  1305696483  Toy Story (1995)   

                                        genres  (no genres listed)  action  \
0  Adventure|Animation|Children|Comedy|Fantasy                   0       0   
1  Adventure|Animation|Children|Comedy|Fantasy                   0       0   
2  Adventure|Animation|Children|Comedy|Fantasy                   0       0   
3  Adventure|Animation|Children|Comedy|Fantasy                   0       0   
4  Adventure|Animation|Children|Comedy|Fantasy                   0       0   

   adventure  animation  ...  film-noir  horror  imax  musical  mystery  \
0          1          1  ...          0       0     0        0        0   
1          1          1  ...          0       0     0        0        0   
2          1          1  ...          0       0     0        0        0   
3          1          1  ...          0       0     0        0        0   
4          1          1  ...          0       0     0        0        0   

   romance  sci-fi  thriller  war  western  
0        0       0         0    0        0  
1        0       0         0    0        0  
2        0       0         0    0        0  
3        0       0         0    0        0  
4        0       0         0    0        0  

[5 rows x 26 columns]

In [14]:
movie_ids = final_df['movieId'].unique()
shuffled_movie_ids = random.sample(list(movie_ids), len(movie_ids))
print("Shuffled movie ids: ", shuffled_movie_ids)

Shuffled movie ids:  [104944, 48150, 26366, 86320, 6188, 103980, 1667, 36401, 97860, 172591, 5684, 965, 131610, 39435, 3363, 688, 1372, 45648, 153386, 33836, 7786, 96726, 2093, 1094, 107, 6984, 7028, 26736, 8933, 90249, 27721, 104879, 7312, 486, 2143, 8685, 157, 85399, 115502, 7306, 1093, 75416, 1103, 86922, 5033, 42718, 4263, 5103, 143511, 112183, 57274, 8979, 1284, 83910, 58291, 141513, 103335, 8376, 163386, 7316, 1380, 8117, 93855, 4887, 2137, 78836, 671, 2098, 2953, 4679, 71494, 5205, 7455, 31422, 5278, 2629, 2362, 1659, 33558, 109374, 27838, 136016, 122627, 2268, 133771, 56757, 65588, 4402, 7121, 7932, 8535, 26765, 8019, 5471, 4241, 4855, 48774, 193565, 92, 7745, 1998, 120783, 5694, 7087, 6584, 33090, 4020, 3141, 6769, 7772, 118706, 172229, 105351, 5480, 56367, 3054, 2403, 151455, 1414, 31225, 60760, 3038, 722, 46976, 170993, 2065, 190207, 5629, 55282, 3033, 59429, 232, 60487, 59915, 1366, 4970, 2921, 6242, 2732, 3525, 804, 45635, 53993, 26861, 5358, 49822, 5673, 1119, 2161, 58975

In [15]:
# this is what the model will be trained on. It will exclude the following columns
train_data = movies_df.drop(columns=['movieId', 'genres', 'title'])

movie_ids = final_df['movieId'].unique()
# find the range of movie_ids
max_movie_id = max(movie_ids)
min_movie_id = min(movie_ids)
print("Max movie id: ", max_movie_id)
print("Min movie id: ", min_movie_id)

Max movie id:  193609
Min movie id:  1


In [16]:
# creating an ultimate dictionary with the following format: {UserId: {rating: [movieId1, movieId2, ...], ...}, ...}
ultimate_dict = {}
for index, row in final_df.iterrows():
    userId = row['userId']
    rating = row['rating']
    movieId = row['movieId']
    if userId not in ultimate_dict:
        ultimate_dict[userId] = {}
    if rating not in ultimate_dict[userId]:
        ultimate_dict[userId][rating] = []
    ultimate_dict[userId][rating].append(movieId)

print(ultimate_dict)

{1: {0.8: [1, 3, 6, 110, 235, 349, 356, 367, 441, 480, 543, 552, 590, 592, 593, 733, 804, 943, 1042, 1060, 1090, 1127, 1208, 1214, 1265, 1348, 1473, 1500, 1552, 1620, 1777, 1793, 1805, 1920, 1967, 2000, 2012, 2028, 2046, 2054, 2096, 2099, 2105, 2143, 2174, 2193, 2268, 2273, 2366, 2406, 2450, 2478, 2492, 2616, 2628, 2640, 2644, 2648, 2797, 2826, 2916, 2985, 2997, 3062, 3168, 3439, 3440, 3479, 3489, 3527, 3617, 3639, 3740, 3744, 3809, 4006], 1.0: [47, 50, 101, 151, 157, 163, 216, 231, 260, 333, 362, 457, 527, 553, 596, 608, 661, 919, 923, 940, 954, 1023, 1024, 1025, 1029, 1031, 1032, 1049, 1073, 1080, 1089, 1092, 1097, 1136, 1196, 1197, 1198, 1206, 1210, 1213, 1220, 1222, 1224, 1226, 1240, 1256, 1270, 1275, 1278, 1282, 1291, 1298, 1517, 1573, 1587, 1617, 1625, 1732, 1804, 1927, 1954, 2005, 2018, 2033, 2048, 2058, 2078, 2090, 2094, 2115, 2116, 2137, 2139, 2141, 2161, 2291, 2329, 2353, 2387, 2395, 2427, 2459, 2470, 2502, 2529, 2542, 2571, 2580, 2596, 2641, 2654, 2692, 2700, 2716, 2761, 285

In [17]:
# Check that each ultimate_dict entry is in the movies_df
for userId in ultimate_dict:
    for rating in ultimate_dict[userId]:
        for movieId in ultimate_dict[userId][rating]:
            if movieId not in movies_df['movieId'].values:
                print("MovieId not in movies_df: ", movieId)

In [18]:
# Create a dictionary of the index to the movieId in the final_df: index_to_movieId
train_index_to_movieId = dict(enumerate(final_df['movieId']))
print("Train Index to MovieId: ", len(train_index_to_movieId.keys()), train_index_to_movieId)

Train Index to MovieId:  100836 {0: 1, 1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1, 16: 1, 17: 1, 18: 1, 19: 1, 20: 1, 21: 1, 22: 1, 23: 1, 24: 1, 25: 1, 26: 1, 27: 1, 28: 1, 29: 1, 30: 1, 31: 1, 32: 1, 33: 1, 34: 1, 35: 1, 36: 1, 37: 1, 38: 1, 39: 1, 40: 1, 41: 1, 42: 1, 43: 1, 44: 1, 45: 1, 46: 1, 47: 1, 48: 1, 49: 1, 50: 1, 51: 1, 52: 1, 53: 1, 54: 1, 55: 1, 56: 1, 57: 1, 58: 1, 59: 1, 60: 1, 61: 1, 62: 1, 63: 1, 64: 1, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 1, 72: 1, 73: 1, 74: 1, 75: 1, 76: 1, 77: 1, 78: 1, 79: 1, 80: 1, 81: 1, 82: 1, 83: 1, 84: 1, 85: 1, 86: 1, 87: 1, 88: 1, 89: 1, 90: 1, 91: 1, 92: 1, 93: 1, 94: 1, 95: 1, 96: 1, 97: 1, 98: 1, 99: 1, 100: 1, 101: 1, 102: 1, 103: 1, 104: 1, 105: 1, 106: 1, 107: 1, 108: 1, 109: 1, 110: 1, 111: 1, 112: 1, 113: 1, 114: 1, 115: 1, 116: 1, 117: 1, 118: 1, 119: 1, 120: 1, 121: 1, 122: 1, 123: 1, 124: 1, 125: 1, 126: 1, 127: 1, 128: 1, 129: 1, 130: 1, 131: 1, 132: 1, 133: 1, 134: 

In [19]:
# Need to normalize the rows of the data
train_data = train_data.div(train_data.sum(axis=1), axis=0)
train_data.head()

(no genres listed)  action  adventure  animation  children    comedy  \
0                 0.0     0.0   0.200000        0.2  0.200000  0.200000   
1                 0.0     0.0   0.333333        0.0  0.333333  0.000000   
2                 0.0     0.0   0.000000        0.0  0.000000  0.500000   
3                 0.0     0.0   0.000000        0.0  0.000000  0.333333   
4                 0.0     0.0   0.000000        0.0  0.000000  1.000000   

   crime  documentary     drama   fantasy  film-noir  horror  imax  musical  \
0    0.0          0.0  0.000000  0.200000        0.0     0.0   0.0      0.0   
1    0.0          0.0  0.000000  0.333333        0.0     0.0   0.0      0.0   
2    0.0          0.0  0.000000  0.000000        0.0     0.0   0.0      0.0   
3    0.0          0.0  0.333333  0.000000        0.0     0.0   0.0      0.0   
4    0.0          0.0  0.000000  0.000000        0.0     0.0   0.0      0.0   

   mystery   romance  sci-fi  thriller  war  western  
0      0.0  0.000000     0.0       0.0  0.0      0.0  
1      0.0  0.000000     0.0       0.0  0.0      0.0  
2      0.0  0.500000     0.0       0.0  0.0      0.0  
3      0.0  0.333333     0.0       0.0  0.0      0.0  
4      0.0  0.000000     0.0       0.0  0.0      0.0

# Creating the model

In [20]:
from sklearn.neighbors import NearestNeighbors

# Initializing the parameters of the model
model = NearestNeighbors(n_neighbors=1000, algorithm='brute')

model.fit(train_data)

NearestNeighbors(algorithm='brute', n_neighbors=1000)

In [21]:
from random import randint

# Any movieId that is at or rated above this threshold will be considered a good movie and included in the average vector fed into KNN model
rating_threshold = 0.8

# pick a random user from the list of users in the ultimate_dict
random_user = list(ultimate_dict.keys())[randint(0, len(ultimate_dict.keys()))]
print("Random User: ", random_user)
print("User's ratings: ", ultimate_dict[random_user])

# Get the movies that the user has rated above the rating_threshold
good_movies = []
watched_movies = []
for rating, movies in ultimate_dict[random_user].items():
    if rating >= rating_threshold:
        good_movies += movies
    watched_movies += movies


# if the user has not rated any movies above the rating_threshold, then pick a random set of 5 movies from the list of movies that the user has not seen
if len(good_movies) == 0:
    good_movies = list(set(train_data.index) - set(ultimate_dict[random_user]))
    good_movies = good_movies[:5]

print("Good Movies: ", len(good_movies), good_movies)
print("Watched Movies: ", len(watched_movies), watched_movies)

Random User:  3
User's ratings:  {0.1: [527, 2018, 2090, 31, 647, 688, 720, 914, 1093, 1124, 1263, 1272, 1302, 2080, 2424, 3210, 3949, 5048, 6238, 72378], 0.7: [1275], 0.9: [1587, 3024, 5764, 7899, 26409], 0.4: [2105], 1.0: [3703, 849, 2851, 4518, 5181, 5746, 5919, 6835, 7991, 70946], 0.6: [1371], 0.8: [2288]}
Good Movies:  16 [1587, 3024, 5764, 7899, 26409, 3703, 849, 2851, 4518, 5181, 5746, 5919, 6835, 7991, 70946, 2288]
Watched Movies:  39 [527, 2018, 2090, 31, 647, 688, 720, 914, 1093, 1124, 1263, 1272, 1302, 2080, 2424, 3210, 3949, 5048, 6238, 72378, 1275, 1587, 3024, 5764, 7899, 26409, 2105, 3703, 849, 2851, 4518, 5181, 5746, 5919, 6835, 7991, 70946, 1371, 2288]


In [23]:
# Cretaing a vector containing the combined ratings of the good movies the user has highly rated
average_vector = np.zeros(len(train_data.columns))
for movieId in good_movies:
    # need row of movies_df that corresponds to the movieId
    row = movies_df[movies_df["movieId"] == movieId]

    # strip away the frist 3 columns
    row = list(row.values[0])[3:]
    # convert row to a numpy array
    row = np.array(row)

    row = row / sum(row)
    average_vector += row

average_vector = average_vector / len(good_movies)
user_vector = average_vector

print("User Vector: ", user_vector)

User Vector:  [0.         0.21875    0.07291667 0.         0.         0.03125
 0.         0.         0.015625   0.05208333 0.         0.19270833
 0.         0.         0.015625   0.         0.296875   0.10416667
 0.         0.        ]


In [24]:
# plugging the user_vector into the KNN model to get the indices of the 1000 most similar movies
distances, indices = model.kneighbors([user_vector])

distances = list(distances[0])

# Find list of movies that the movie_index correspond to
recommended_movies = []
for index in indices[0]:
    recommended_movies.append(train_index_to_movieId[index])

for index, movie in enumerate(watched_movies):
    if movie in recommended_movies:
        recommended_movies.remove(movie)
        distances.remove(distances[index])

recommended_movie_names = []
for movie in recommended_movies:
    recommended_movie_names.append(movies_df[movies_df['movieId'] == movie]['title'].values[0])

num_pintouts = 5
print("Recommended movies for user: ", random_user)
for i in range(num_pintouts):
    print("Recommendation ", i + 1, ": ", recommended_movie_names[i], " with distance: ", distances[i])

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(


Recommended movies for user:  3
Recommendation  1 :  Willy Wonka & the Chocolate Factory (1971)  with distance:  0.19291936643179308
Recommendation  2 :  Twister (1996)  with distance:  0.19291936643179308
Recommendation  3 :  Pulp Fiction (1994)  with distance:  0.19291936643179308
Recommendation  4 :  Sword in the Stone, The (1963)  with distance:  0.19291936643179308
Recommendation  5 :  E.T. the Extra-Terrestrial (1982)  with distance:  0.19291936643179308
